# Situation Flow Notebook (WIP)

Standalone notebook to manage situations using the same data submodules (`data/`, `situations/`).


## Setup
- Ensure submodules are pulled: `!git submodule update --init --recursive`
- Install deps: `pip install ipywidgets openai`
- The notebook reads/writes the existing `data/gloss/{lang}/*.json` files.


In [2]:
from __future__ import annotations

import json
import os
import re
from dataclasses import dataclass, field
from pathlib import Path
from typing import Any

import ipywidgets as widgets
from IPython.display import display

try:
    from openai import OpenAI
except Exception as exc:  # noqa: BLE001
    raise RuntimeError("Install the openai package: pip install openai") from exc

client = OpenAI()  # expects OPENAI_API_KEY in env

# Adjust if you move the notebook; assumes /notebooks relative to repo root.
repo_root = Path.cwd().resolve().parent
DATA_ROOT = repo_root / "data"
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Expected data directory at {DATA_ROOT}")


In [3]:
# --- Gloss model ---

def normalize_language_code(code: str | None) -> str:
    return (code or "").strip().lower()


def derive_slug(text: str) -> str:
    text = (text or "").strip().lower()
    text = re.sub(r"[^a-z0-9]+", "-", text)
    text = re.sub(r"-+", "-", text).strip("-")
    return text


@dataclass
class Gloss:
    content: str
    language: str = "und"
    transcriptions: dict[str, str] = field(default_factory=dict)
    logs: dict[str, str] = field(default_factory=dict)
    morphologically_related: list[str] = field(default_factory=list)
    parts: list[str] = field(default_factory=list)
    has_similar_meaning: list[str] = field(default_factory=list)
    sounds_similar: list[str] = field(default_factory=list)
    usage_examples: list[str] = field(default_factory=list)
    to_be_differentiated_from: list[str] = field(default_factory=list)
    collocations: list[str] = field(default_factory=list)
    typical_follow_up: list[str] = field(default_factory=list)
    children: list[str] = field(default_factory=list)
    translations: list[str] = field(default_factory=list)
    notes: list[str] = field(default_factory=list)
    tags: list[str] = field(default_factory=list)
    slug: str | None = None

    @classmethod
    def from_dict(cls, data: dict[str, Any], slug: str | None = None, language: str | None = None) -> "Gloss":
        return cls(
            content=data.get("content", ""),
            language=normalize_language_code(language or data.get("language", "und")),
            transcriptions=dict(data.get("transcriptions", {}) or {}),
            logs=dict(data.get("logs", {}) or {}),
            morphologically_related=list(data.get("morphologically_related", []) or []),
            parts=list(data.get("parts", []) or []),
            has_similar_meaning=list(data.get("has_similar_meaning", []) or []),
            sounds_similar=list(data.get("sounds_similar", []) or []),
            usage_examples=list(data.get("usage_examples", []) or []),
            to_be_differentiated_from=list(data.get("to_be_differentiated_from", []) or []),
            collocations=list(data.get("collocations", []) or []),
            typical_follow_up=list(data.get("typical_follow_up", []) or []),
            children=list(data.get("children", []) or []),
            translations=list(data.get("translations", []) or []),
            notes=list(data.get("notes", []) or []),
            tags=list(data.get("tags", []) or []),
            slug=slug,
        )

    def to_dict(self) -> dict[str, Any]:
        return {
            "content": self.content,
            "language": normalize_language_code(self.language),
            "transcriptions": self.transcriptions,
            "logs": self.logs,
            "morphologically_related": self.morphologically_related,
            "parts": self.parts,
            "has_similar_meaning": self.has_similar_meaning,
            "sounds_similar": self.sounds_similar,
            "usage_examples": self.usage_examples,
            "to_be_differentiated_from": self.to_be_differentiated_from,
            "collocations": self.collocations,
            "typical_follow_up": self.typical_follow_up,
            "children": self.children,
            "translations": self.translations,
            "notes": self.notes,
            "tags": self.tags,
        }


In [4]:
# --- Storage helpers (filesystem-backed, same layout as the app) ---

RELATIONSHIP_FIELDS = [
    "morphologically_related",
    "parts",
    "has_similar_meaning",
    "sounds_similar",
    "usage_examples",
    "to_be_differentiated_from",
    "collocations",
    "typical_follow_up",
    "children",
    "translations",
    "notes",
    "tags",
]


class GlossStorage:
    def __init__(self, data_root: Path):
        self.data_root = Path(data_root)
        self.gloss_root = self.data_root / "gloss"
        if not self.gloss_root.exists():
            raise FileNotFoundError(f"Gloss directory not found: {self.gloss_root}")

    def _language_dir(self, language: str) -> Path:
        lang = normalize_language_code(language)
        target = self.gloss_root / lang
        target.mkdir(parents=True, exist_ok=True)
        return target

    def _path_for(self, language: str, slug: str) -> Path:
        return self._language_dir(language) / f"{slug}.json"

    def list_glosses(self) -> list[Gloss]:
        glosses: list[Gloss] = []
        if not self.gloss_root.exists():
            return glosses
        for language_dir in sorted(self.gloss_root.iterdir()):
            if not language_dir.is_dir():
                continue
            for gloss_file in sorted(language_dir.glob("*.json")):
                with gloss_file.open("r", encoding="utf-8") as handle:
                    data = json.load(handle)
                gloss = Gloss.from_dict(data, slug=gloss_file.stem, language=language_dir.name)
                glosses.append(gloss)
        return glosses

    def load_gloss(self, language: str, slug: str) -> Gloss | None:
        path = self._path_for(language, slug)
        if not path.exists():
            return None
        with path.open("r", encoding="utf-8") as handle:
            data = json.load(handle)
        return Gloss.from_dict(data, slug=slug, language=language)

    def find_gloss_by_slug(self, language: str, slug: str) -> Gloss | None:
        return self.load_gloss(normalize_language_code(language), slug)

    def create_gloss(self, gloss: Gloss) -> Gloss:
        slug = derive_slug(gloss.content)
        if not slug:
            raise ValueError("Content must produce a valid slug.")
        language = normalize_language_code(gloss.language)
        target = self._path_for(language, slug)
        if target.exists():
            raise FileExistsError(f"Gloss already exists: {language}:{slug}")
        self._write_gloss(target, gloss)
        gloss.slug = slug
        gloss.language = language
        return gloss

    def save_gloss(self, gloss: Gloss) -> Gloss:
        if not gloss.slug or not gloss.language:
            raise ValueError("Gloss must have language and slug before saving.")
        target = self._path_for(gloss.language, gloss.slug)
        self._write_gloss(target, gloss)
        return gloss

    def ensure_gloss(self, language: str, content: str) -> Gloss:
        language = normalize_language_code(language)
        existing = self.find_gloss_by_content(language, content)
        if existing:
            return existing
        new_gloss = Gloss(content=content, language=language)
        return self.create_gloss(new_gloss)

    def find_gloss_by_content(self, language: str, content: str) -> Gloss | None:
        language = normalize_language_code(language)
        slug = derive_slug(content)
        if not slug:
            return None
        return self.load_gloss(language, slug)

    def resolve_reference(self, ref: str) -> Gloss | None:
        if ":" not in ref:
            return None
        language, slug = ref.split(":", 1)
        language = normalize_language_code(language)
        slug = slug.strip()
        if not slug:
            return None
        return self.load_gloss(language, slug)

    def _write_gloss(self, path: Path, gloss: Gloss) -> None:
        payload = gloss.to_dict()
        with path.open("w", encoding="utf-8") as handle:
            json.dump(payload, handle, indent=2, ensure_ascii=False)


def attach_relation(storage: GlossStorage, source: Gloss, field: str, target: Gloss) -> None:
    if field not in RELATIONSHIP_FIELDS:
        raise ValueError(f"Unknown relationship field: {field}")
    refs = getattr(source, field, []) or []
    ref = f"{target.language}:{target.slug or derive_slug(target.content)}"
    if ref not in refs:
        refs = list(refs) + [ref]
        setattr(source, field, refs)
        storage.save_gloss(source)


In [5]:
# --- Situation logic (adapted from sbll_cms/situations_logic.py) ---

SPLIT_LOG_MARKER = "SPLIT_CONSIDERED_UNNECESSARY"
TRANSLATION_IMPOSSIBLE_MARKER = "TRANSLATION_CONSIDERED_IMPOSSIBLE"
USAGE_IMPOSSIBLE_MARKER = "USAGE_EXAMPLE_CONSIDERED_IMPOSSIBLE"


def paraphrase_display(gloss: Gloss) -> str:
    text = gloss.content or gloss.slug or ""
    if gloss.slug and gloss.slug not in text:
        text = f"{text} ({gloss.slug})"
    return text


def build_goal_nodes(situation: Gloss, storage: GlossStorage, native_language: str, target_language: str):
    stats = {
        "situation_glosses": set(),
        "glosses_to_learn": set(),
        "native_missing": set(),
        "target_missing": set(),
        "parts_missing": set(),
        "usage_missing": set(),
        "gloss_map": {},
    }
    seen_keys: set[str] = set()
    nodes = []

    def gloss_key(gl):
        return f"{gl.language}:{gl.slug or gl.content}"

    def has_log(gl, marker: str) -> bool:
        logs = getattr(gl, "logs", {}) or {}
        if not isinstance(logs, dict):
            return False
        return any(marker in str(val) for val in logs.values())

    def has_translation(gl, lang: str) -> bool:
        return any(ref.startswith(f"{lang}:") for ref in (gl.translations or []))

    def mark_stats(gl, usage_lineage: bool, parts_line: bool, learn_lang: str):
        key = gloss_key(gl)
        stats["gloss_map"][key] = gl
        stats["situation_glosses"].add(key)

        if not (getattr(gl, "parts", None) or []) and not has_log(gl, SPLIT_LOG_MARKER):
            stats["parts_missing"].add(key)

        if gl.language == target_language:
            if not has_translation(gl, native_language) and not has_log(gl, f"{TRANSLATION_IMPOSSIBLE_MARKER}:{native_language}"):
                stats["native_missing"].add(key)
            if not usage_lineage and not has_log(gl, f"{USAGE_IMPOSSIBLE_MARKER}:{target_language}") and not (gl.usage_examples or []):
                stats["usage_missing"].add(key)
        elif gl.language == native_language:
            if not has_translation(gl, target_language) and not has_log(gl, f"{TRANSLATION_IMPOSSIBLE_MARKER}:{target_language}"):
                stats["target_missing"].add(key)

        if parts_line and gl.language == learn_lang:
            stats["glosses_to_learn"].add(key)

        return {
            "warn_native_missing": key in stats["native_missing"],
            "warn_target_missing": key in stats["target_missing"],
            "warn_usage_missing": key in stats["usage_missing"],
        }

    def build_node(gloss, role="root", marker="", usage_lineage=False, allow_translations=True, path=None, parts_line=False, learn_lang=""):
        tags = gloss.tags or []
        if gloss.language == target_language and "eng:paraphrase" in tags:
            return None

        path = set(path or [])
        key = gloss_key(gloss)
        seen_keys.add(key)

        flags = mark_stats(gloss, usage_lineage, parts_line, learn_lang)

        node = {
            "gloss": gloss,
            "children": [],
            "marker": marker,
            "bold": parts_line and gloss.language == learn_lang,
            "role": role,
            "warn_native_missing": flags["warn_native_missing"],
            "warn_target_missing": flags["warn_target_missing"],
            "warn_usage_missing": flags["warn_usage_missing"],
            "warn_parts_missing": key in stats["parts_missing"],
        }

        if key in path:
            return node
        next_path = set(path or [])
        next_path.add(key)

        if role in ("root", "part", "usage_part"):
            stats["glosses_to_learn"].add(key)

        for part_ref in getattr(gloss, "parts", []):
            part_gloss = storage.resolve_reference(part_ref)
            if not part_gloss:
                continue
            child_parts_line = parts_line if role == "root" else False
            part_node = build_node(
                part_gloss,
                role="usage_part" if role in ("usage", "usage_part") else "part",
                usage_lineage=usage_lineage,
                allow_translations=True,
                path=next_path,
                parts_line=child_parts_line,
                learn_lang=learn_lang,
            )
            if part_node:
                node["children"].append(part_node)

        if allow_translations:
            other_lang = None
            if gloss.language == native_language and target_language:
                other_lang = target_language
            elif gloss.language == target_language and native_language:
                other_lang = native_language
            if other_lang:
                for ref in gloss.translations or []:
                    ref_lang = ref.split(":", 1)[0].strip().lower()
                    if ref_lang != other_lang.lower():
                        continue
                    t_gloss = storage.resolve_reference(ref)
                    if not t_gloss:
                        continue
                    child_key = gloss_key(t_gloss)
                    t_node = build_node(
                        t_gloss,
                        role="translation",
                        marker="",
                        usage_lineage=usage_lineage,
                        allow_translations=child_key not in next_path,
                        path=next_path,
                        parts_line=False,
                        learn_lang=learn_lang,
                    )
                    if t_node:
                        node["children"].append(t_node)

        if gloss.language == target_language and not usage_lineage:
            if gloss.usage_examples:
                for u_ref in getattr(gloss, "usage_examples", []):
                    u_gloss = storage.resolve_reference(u_ref)
                    if not u_gloss:
                        continue
                    usage_node = build_node(
                        u_gloss,
                        role="usage",
                        marker="USG ",
                        usage_lineage=True,
                        allow_translations=True,
                        path=next_path,
                        parts_line=False,
                        learn_lang=learn_lang,
                    )
                    if usage_node:
                        node["children"].append(usage_node)

        return node

    for ref in situation.children:
        gloss = storage.resolve_reference(ref)
        if not gloss:
            continue
        tags = gloss.tags or []
        marker = ""
        if gloss.language == native_language and "eng:procedural-paraphrase-expression-goal" in tags:
            marker = "PROC "
            learn_lang = native_language
            goal_type = "procedural"
        elif gloss.language == target_language and "eng:understand-expression-goal" in tags:
            marker = "UNDR "
            learn_lang = target_language
            goal_type = "understand"
        else:
            continue
        node = build_node(
            gloss,
            role="root",
            marker=marker,
            usage_lineage=False,
            allow_translations=True,
            parts_line=True,
            learn_lang=learn_lang,
        )
        if node:
            node["goal_type"] = goal_type
            nodes.append(node)
    return nodes, stats


def render_tree(nodes: list[dict[str, Any]]):
    lines: list[str] = []

    def label_for(node):
        gloss = node["gloss"]
        text = paraphrase_display(gloss)
        markers_after = ""
        if node.get("warn_native_missing") or node.get("warn_target_missing"):
            markers_after += " [WARN-TRANSLATION]"
        if node.get("warn_usage_missing"):
            markers_after += " [WARN-USAGE]"
        if node.get("warn_parts_missing"):
            markers_after += " [WARN-PARTS]"
        content = f"{text}{markers_after}"
        if node.get("bold"):
            content = f"**{content}**"
        return content

    def walk(node_list, prefix=""):
        total = len(node_list)
        for idx, node in enumerate(node_list):
            is_last = idx == total - 1
            connector = "`-- " if is_last else "|-- "
            lines.append(f"{prefix}{connector}{node.get('marker', '')}{label_for(node)}")
            if node.get("children"):
                walk(node["children"], f"{prefix}{'    ' if is_last else '|   '}")

    walk(nodes)
    return lines


def collect_situation_stats(storage: GlossStorage, situation: Gloss, native_language: str, target_language: str):
    _nodes, stats = build_goal_nodes(
        situation,
        storage=storage,
        native_language=native_language,
        target_language=target_language,
    )
    return stats


In [6]:
# --- Load glosses and derive situation list ---

storage = GlossStorage(DATA_ROOT)
all_glosses = storage.list_glosses()
print(f"Loaded {len(all_glosses)} glosses from {DATA_ROOT}")

situations = [g for g in all_glosses if "eng:situation" in (g.tags or [])]
lang_codes = sorted({g.language for g in all_glosses})
if not situations:
    print("No situations found. Check that data submodule is present.")


Loaded 492 glosses from /home/brokkoli/GITHUB/glosses4learning-cms/data


In [7]:
# --- Manager view: pick situation + languages, then open flow ---

selection = {"situation_ref": None, "native_language": None, "target_language": None}

situation_options = [
    (f"{g.content} [{g.language}:{g.slug}]", f"{g.language}:{g.slug}") for g in situations
]

situation_dd = widgets.Dropdown(options=situation_options, description="Situation")
native_dd = widgets.Dropdown(options=lang_codes, description="Native")
target_dd = widgets.Dropdown(options=lang_codes, description="Target")
open_btn = widgets.Button(description="Open detailed flow", button_style="primary")
status_out = widgets.Output()
stats_out = widgets.Output()
tree_out = widgets.Output()
flow_box = widgets.VBox([widgets.HTML("<em>No situation selected.</em>")])


def refresh_view():
    tree_out.clear_output()
    stats_out.clear_output()
    if not selection["situation_ref"] or not selection["native_language"] or not selection["target_language"]:
        flow_box.children = [widgets.HTML("<em>Select a situation and languages.</em>")]
        return
    situation = storage.resolve_reference(selection["situation_ref"])
    if not situation:
        flow_box.children = [widgets.HTML(f"<b>Missing situation</b>: {selection['situation_ref']}")]
        return
    nodes, stats = build_goal_nodes(
        situation,
        storage=storage,
        native_language=selection["native_language"],
        target_language=selection["target_language"],
    )
    lines = render_tree(nodes)
    with tree_out:
        display(widgets.HTML(f"<pre>{'\\n'.join(lines) or '(no tree)'}</pre>"))
    with stats_out:
        summary = {
            "parts_missing": len(stats.get("parts_missing", [])),
            "native_missing": len(stats.get("native_missing", [])),
            "target_missing": len(stats.get("target_missing", [])),
            "usage_missing": len(stats.get("usage_missing", [])),
            "glosses_to_learn": len(stats.get("glosses_to_learn", [])),
        }
        print("Stats:", summary)
    header = widgets.HTML(
        f"<b>Flow for</b> {situation.content} &nbsp;|&nbsp; Native: {selection['native_language']} &nbsp; Target: {selection['target_language']}"
    )
    todo = widgets.HTML("<em>Tool panels go here (generate/accept goals, translations, splits, examples).</em>")
    flow_box.children = [header, stats_out, tree_out, todo]


def on_open_clicked(_):
    selection["situation_ref"] = situation_dd.value
    selection["native_language"] = native_dd.value
    selection["target_language"] = target_dd.value
    with status_out:
        status_out.clear_output()
        print(
            f"Selected {selection['situation_ref']} | native={selection['native_language']} | target={selection['target_language']}"
        )
    refresh_view()


open_btn.on_click(on_open_clicked)

manager_box = widgets.VBox([
    widgets.HTML("<h3>Situation manager</h3>"),
    situation_dd,
    widgets.HBox([native_dd, target_dd]),
    open_btn,
    status_out,
])

display(manager_box)
display(flow_box)
